In [38]:
import pennylane as qml
from pennylane import numpy as np
import csv

np.random.seed(42)

In [39]:
n_wires = 8
graph = [(0,1), (0,6), (0,7), (1,2), (1,6), (2,4), (2,7), (3,7), (3,6), (4,5), (4,6), (1,5), (5,6), (5,7), (6,7)]

# unitary operator U_B with parameter beta
def U_B(beta):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)


# unitary operator U_C with parameter gamma
def U_C(gamma):
    for edge in graph:
        wire1 = edge[0]
        wire2 = edge[1]
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(gamma, wires=wire2)
        qml.CNOT(wires=[wire1, wire2])

In [40]:
def bitstring_to_int(bit_string_sample):
    bit_string = "".join(str(bs) for bs in bit_string_sample)
    return int(bit_string, base=2)

In [41]:
dev = qml.device("default.qubit", wires=n_wires, shots=1)

In [42]:
@qml.qnode(dev)
def circuit(gammas, betas, edge=None, n_layers=1):
    # apply Hadamards to get the n qubit |+> state
    for wire in range(n_wires):
        qml.Hadamard(wires=wire)
    # p instances of unitary operators
    for i in range(n_layers):
        U_C(gammas[i])
        U_B(betas[i])
    if edge is None:
        # measurement phase
        return qml.sample()
    # during the optimization phase we are evaluating a term
    # in the objective using expval
    H = qml.PauliZ(edge[0]) @ qml.PauliZ(edge[1])
    return qml.expval(H)

In [43]:
def hamiltonian(config, graph):
    
    H = 0
    for i, j in graph:
        
        zi = config[i]
        zj = config[j]
        H += 0.5 * (1 - zi*zj)
    
    return H

In [44]:
def qaoa_maxcut(n_layers=1):
    print("\np={:d}".format(n_layers))
    score = []
    # initialize the parameters near zero
    init_params = 0.01 * np.random.rand(2, n_layers, requires_grad=True)

    # minimize the negative of the objective function
    def objective(params):
        gammas = params[0]
        betas = params[1]
        neg_obj = 0
        for edge in graph:
            # objective for the MaxCut problem
            neg_obj -= 0.5 * (1 - circuit(gammas, betas, edge=edge, n_layers=n_layers))
        return neg_obj

    # initialize optimizer: Adagrad works well empirically
    opt = qml.AdagradOptimizer(stepsize=0.5)

    # optimize parameters in objective
    params = init_params
    steps = 30
    for i in range(steps):
        params = opt.step(objective, params)
        sco = -objective(params)
        score.append(sco)
        with open ('QAOARandomScore1.csv', 'w', newline='') as file1:
            writer = csv.writer(file1)
            writer.writerow(score)
        print("Objective after step {:5d}: {: .7f}".format(i+1, sco))
        #if (i + 1) % 5 == 0:
        #    print("Objective after step {:5d}: {: .7f}".format(i + 1, -objective(params)))

    # sample measured bitstrings 100 times
    bit_strings = []
    n_samples = 100
    for i in range(0, n_samples):
        bit_strings.append(bitstring_to_int(circuit(params[0], params[1], edge=None, n_layers=n_layers)))

    simple_measurements = []
    for i in range(0, n_samples):
        measurements = circuit(params[0], params[1], edge=None, n_layers=n_layers)
        simple_measurements.append(measurements)
        bit_strings.append(bitstring_to_int(measurements))
        
    samples_energies = []
    for m in simple_measurements:
        m = m.tolist()
        m = [int(x) if int(x)==1 else -1 for x in m]
        samples_energies.append(hamiltonian(m, graph))
       
    average_energy = sum(samples_energies)/len(samples_energies)
    energy = []
    energy.append(average_energy)
    print('average energy:', average_energy)
    with open ('QAOARandomEnergy1.csv', 'w', newline='') as file2:
        writer = csv.writer(file2)
        writer.writerow(energy)

    # print optimal parameters and most frequently sampled bitstring
    counts = np.bincount(np.array(bit_strings))
    most_freq_bit_string = np.argmax(counts)
    print("Optimized (gamma, beta) vectors:\n{}".format(params[:, :n_layers]))
    print("Most frequently sampled bit string is: {:04b}".format(most_freq_bit_string))

    return -objective(params), bit_strings


# perform qaoa on our graph with p=1,2 and
# keep the bitstring sample lists
bitstrings1 = qaoa_maxcut(n_layers=4)[1]



p=4
Objective after step     1:  6.0000000
Objective after step     2:  4.0000000
Objective after step     3:  6.0000000
Objective after step     4:  8.0000000
Objective after step     5:  8.0000000
Objective after step     6:  7.0000000
Objective after step     7:  7.0000000
Objective after step     8:  11.0000000
Objective after step     9:  10.0000000
Objective after step    10:  10.0000000
Objective after step    11:  6.0000000
Objective after step    12:  9.0000000
Objective after step    13:  7.0000000
Objective after step    14:  6.0000000
Objective after step    15:  7.0000000
Objective after step    16:  7.0000000
Objective after step    17:  6.0000000
Objective after step    18:  8.0000000
Objective after step    19:  8.0000000
Objective after step    20:  9.0000000
Objective after step    21:  3.0000000
Objective after step    22:  6.0000000
Objective after step    23:  7.0000000
Objective after step    24:  8.0000000
Objective after step    25:  8.0000000
Objective after s